In [15]:
import os
from pathlib import Path

while Path.cwd().name.lower() != "aiml25-exam" and "aiml25-exam" in str(Path.cwd()).lower():
    os.chdir("..")  # Move up one directory
print(f"Working directory set to: {Path.cwd()}")

Working directory set to: /Users/emilstausbol/Documents/GitHub/AIML25-Exam


In [16]:
from src.yolo.yolo import Yolo
from ultralytics import YOLO
from src.utils.path import from_root

In [18]:
model = YOLO(from_root("models/yolo-trained.pt"))

yolo = Yolo(model)
yolo.predict(from_root("datasets/test/images/1.png"))


image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/datasets/test/images/1.png: 448x640 14 sub-flows, 1256.5ms
Speed: 17.4ms preprocess, 1256.5ms inference, 12.0ms postprocess per image at shape (1, 3, 448, 640)


In [19]:
bboxes = yolo.getBBoxes()
bboxes

[{'xywh': [1163.42822265625,
   383.06591796875,
   1180.1429443359375,
   601.815185546875],
  'confidence': 0.91915363073349,
  'class_id': 0,
  'xyxy': [573.3568115234375,
   82.1583251953125,
   1753.499755859375,
   683.9735107421875]},
 {'xywh': [616.860595703125,
   920.391357421875,
   646.033935546875,
   607.21728515625],
  'confidence': 0.8993555903434753,
  'class_id': 0,
  'xyxy': [293.8435974121094, 616.78271484375, 939.8775634765625, 1224.0]},
 {'xywh': [541.1039428710938,
   578.2733154296875,
   619.6742553710938,
   503.87139892578125],
  'confidence': 0.8913698196411133,
  'class_id': 0,
  'xyxy': [231.2667999267578,
   326.33758544921875,
   850.9410400390625,
   830.208984375]},
 {'xywh': [1039.62646484375,
   565.8004150390625,
   388.488037109375,
   481.3470764160156],
  'confidence': 0.8903194665908813,
  'class_id': 0,
  'xyxy': [845.3824462890625,
   325.1268615722656,
   1233.8704833984375,
   806.4739379882812]},
 {'xywh': [404.9783630371094,
   175.6228179

In [20]:

# Importing Image class from PIL module
from PIL import Image

for bbox in bboxes:
    im = Image.open(from_root("datasets/test/images/1.png"))
    [left, top, right, bottom] = bbox["xyxy"]
    im1 = im.crop((left, top, right, bottom))
    im1.show()